In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import shutil

# Define the image directory paths
early_blight_dir = r"C:\Users\Avula Jhansy\Downloads\plant_500\Potato___Early_blight"
late_blight_dir = r"C:\Users\Avula Jhansy\Downloads\plant_500\Potato___Late_blight"
healthy_dir = r"C:\Users\Avula Jhansy\Downloads\plant_500\Potato___healthy"

# Target directories where you want to save the balanced dataset
target_dir = r"C:\Users\Avula Jhansy\Downloads\plant_500\500_plant"
os.makedirs(target_dir, exist_ok=True)

# Helper function to copy a random subset of files
def copy_random_subset(source_dir, target_dir, num_images):
    all_images = os.listdir(source_dir)
    selected_images = np.random.choice(all_images, num_images, replace=False)
    os.makedirs(target_dir, exist_ok=True)
    for img in selected_images:
        shutil.copy(os.path.join(source_dir, img), os.path.join(target_dir, img))

# Copy 500 images from Early_blight and Late_blight to the target directory
copy_random_subset(early_blight_dir, os.path.join(target_dir, "Early_blight"), 500)
copy_random_subset(late_blight_dir, os.path.join(target_dir, "Late_blight"), 500)

# Augment Healthy class images to reach 500
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the 152 healthy images
healthy_images = [os.path.join(healthy_dir, img) for img in os.listdir(healthy_dir)]
healthy_images = healthy_images[:152]

# Augment Healthy images
augmented_dir = os.path.join(target_dir, "Healthy")
os.makedirs(augmented_dir, exist_ok=True)

for i, img_path in enumerate(healthy_images):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = x.reshape((1,) + x.shape)
    
    # Generate batches of augmented images
    j = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=augmented_dir, save_prefix=f"healthy_aug_{i}", save_format='png'):
        j += 1
        if j >= (500 - 152) / 152:  # Generate just enough to make total 500 images
            break
